<a href="https://colab.research.google.com/github/Ishgar14/Sentiment-Analysis---Major-Project/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vaderSentiment
!pip install contractions

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [3]:
text = 'I love games'
print(vs.polarity_scores(text))
print(vs.polarity_scores("I'm okay with movies"))

{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'compound': 0.2263}


In [4]:
import pandas as pd
import numpy as np
import os, re
from sklearn import tree, model_selection, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# DATADIR = '/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv'
DATADIR = '/content/drive/MyDrive/Colab Notebooks/testdata.manual.2009.06.14.csv'

col_names = ['Polarity', 'ID', 'Date', 'Query', 'User', 'Tweet']

df = pd.read_csv(DATADIR, encoding='latin-1', header=0, names=col_names)
df = df.drop(['ID', 'Date', 'Query', 'User'], axis=1)  # I dont want date, user, etc now so I'll remove them
df

,Polarity,Tweet
0,4,Reading my kindle2... Love it... Lee childs i...
1,4,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,@kenburbary You'll love your Kindle2. I've had...
3,4,@mikefish Fair enough. But i have the Kindle2...
4,4,@richardebaker no. it is too big. I'm quite ha...
...,...,...
492,2,Ask Programming: LaTeX or InDesign?: submitted...
493,0,"On that note, I hate Word. I hate Pages. I hat..."
494,4,Ahhh... back in a *real* text editing environm...
495,0,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [6]:
def filter_tags(s: str) -> str:
    return re.sub(r'@[a-z]+\s', '', s)

df['Tweet'] = df['Tweet'].apply(filter_tags)

In [7]:
from string import punctuation as pun
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

stopword_list.remove('no')
stopword_list.remove('not')
print("The stopwords are ", stopword_list)
print("The punctuation symbols are ", pun)

def stopwords_removal(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_tokens = [token for token in filtered_tokens if token not in pun and len(token) > 2]
  filtered_tokens = [token for token in filtered_tokens if token == ' ' or token.isalnum()]
#   print(filtered_tokens)
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

df['Tweet'] = df['Tweet'].apply(lambda x: stopwords_removal(x))
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
The stopwords are  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how'

,Polarity,Tweet
0,4,Reading kindle2 Love Lee childs good read
1,4,first assesment fucking rocks
2,4,You love mine months never looked The new big ...
3,4,Fair But Kindle2 think perfect
4,4,quite happy Kindle2
...,...,...
492,2,Ask Programming LaTeX InDesign submitted calci...
493,0,note hate hate hate There said hate All TEXN3R...
494,4,Ahhh back text editing LaTeX
495,0,Trouble Iran Iran far


In [8]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stemmer=PorterStemmer()

def stemming(text):
  updated=[]
  str=nltk.word_tokenize(text)
  for word in str:
   updated.append(stemmer.stem(word))
  sentence=' '.join(updated)
  return sentence

df['Tweet'] = df['Tweet'].apply(lambda x: stemming(x))
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Polarity,Tweet
0,4,read kindle2 love lee child good read
1,4,first asses fuck rock
2,4,you love mine month never look the new big one...
3,4,fair but kindle2 think perfect
4,4,quit happi kindle2
...,...,...
492,2,ask program latex indesign submit calcio1 link...
493,0,note hate hate hate there said hate all texn3r...
494,4,ahhh back text edit latex
495,0,troubl iran iran far


In [9]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
  str=nltk.word_tokenize(text)
  updated=[]
  for word in str:
    updated.append(lemmatizer.lemmatize(word))
  sentence=' '.join(updated)
  return sentence

df['Tweet'] = df['Tweet'].apply(lambda x: lemmatize(x))
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Polarity,Tweet
0,4,read kindle2 love lee child good read
1,4,first ass fuck rock
2,4,you love mine month never look the new big one...
3,4,fair but kindle2 think perfect
4,4,quit happi kindle2
...,...,...
492,2,ask program latex indesign submit calcio1 link...
493,0,note hate hate hate there said hate all texn3r...
494,4,ahhh back text edit latex
495,0,troubl iran iran far


In [10]:
df['compound'] = df['Tweet'].apply(lambda x: (vs.polarity_scores(x)['compound']))
df['vader analysis'] = df['compound'].apply(lambda x: 'pos' if x >= 0 else 'neg')

# Final result is what i will be using for training
df['Final Result']=df['compound'].apply(lambda x: 'pos' if x > 0.5 else 'neu' if 0 <= x <= 0.5 else 'neg')

df

,Polarity,Tweet,compound,vader analysis,Final Result
0,4,read kindle2 love lee child good read,0.7964,pos,pos
1,4,first ass fuck rock,-0.7906,neg,neg
2,4,you love mine month never look the new big one...,0.7579,pos,pos
3,4,fair but kindle2 think perfect,0.7717,pos,pos
4,4,quit happi kindle2,0.0000,pos,neu
...,...,...,...,...,...
492,2,ask program latex indesign submit calcio1 link...,0.0000,pos,neu
493,0,note hate hate hate there said hate all texn3r...,-0.9661,neg,neg
494,4,ahhh back text edit latex,0.0000,pos,neu
495,0,troubl iran iran far,0.0000,pos,neu


In [11]:
from sklearn.model_selection import train_test_split

x=df['Tweet']
y=df['Final Result']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

text_model=Pipeline([('tfid',TfidfVectorizer()),('model',SVC())])

In [13]:
text_model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', de

In [14]:
y_pred = text_model.predict(x_test)
y_pred

array(['neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'pos', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neg',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'pos', 'neu',
       'neu', 'neu', 'neg', 'neu', 'neu', 'neu', 'neg', 'neu', 'neg',
       'pos', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'pos', 'neu', 'neu', 'neu', 'neu', 'pos', 'neu', 'neu',
       'pos', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neg', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu',
       'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neu', 'neg',
       'neg', 'neu', 'neu', 'neu', 'pos', 'pos', 'neu', 'neg', 'neu',
       'neu', 'neu',

In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test) * 100)
print(classification_report(y_pred,y_test))

[[ 7  4  0]
 [26 81 24]
 [ 1  0  7]]
63.33333333333333
              precision    recall  f1-score   support

         neg       0.21      0.64      0.31        11
         neu       0.95      0.62      0.75       131
         pos       0.23      0.88      0.36         8

    accuracy                           0.63       150
   macro avg       0.46      0.71      0.47       150
weighted avg       0.86      0.63      0.70       150



In [16]:
!pip install streamlit
!pip install pyngrok==4.1.1

In [17]:
import streamlit as st

st.title('Sentiment Analysis')

2021-03-08 14:23:17.538 WARNING root: 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [18]:
%%writefile app.py
import streamlit as st
st.title('Sentiment Analysis')
import pandas as pd
import re

def special_removal(text):
  pattern=r'[^A-Za-z\s]'
  text=re.sub(pattern,' ',text)
  return text

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')

stopword_list.remove('no')
stopword_list.remove('not')

def stopwords_removal(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_tokens = [token for token in filtered_tokens if token not in pun and len(token) > 2]
  filtered_tokens = [token for token in filtered_tokens if token == ' ' or token.isalnum()]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stemmer=PorterStemmer()

def stemming(text):
  updated=[]
  str=nltk.word_tokenize(text)
  for word in str:
   updated.append(stemmer.stem(word))
  sentence=' '.join(updated)
  return sentence

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def lemmatize(text):
  str=nltk.word_tokenize(text)
  updated=[]
  for word in str:
    updated.append(lemmatizer.lemmatize(word))
  sentence=' '.join(updated)
  return sentence

df=pd.read_csv('https://github.com/Ishgar14/Sentiment-Analysis---Major-Project/blob/main/testdata.manual.2009.06.14.csv')
x=df['Tweet']
y=df['Final Result']

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model=Pipeline([('tfid',TfidfVectorizer()),('model',SVC())])
text_model.fit(x,y)

select=st.text_input('Enter your msg')
select=special_removal(select)
select=stopwords_removal(select)
select=stemming(select)
select=lemmatize(select)
op=text_model.predict([select])
st.title(op)

Writing app.py


In [19]:
!nohup streamlit run app.py & 
from pyngrok import ngrok 
url = ngrok.connect(port='8051')
url

nohup: appending output to 'nohup.out'


2021-03-08 14:23:19.817 INFO    pyngrok.process: ngrok process starting: 440
2021-03-08 14:23:19.886 INFO    pyngrok.process: t=2021-03-08T14:23:19+0000 lvl=info msg="no configuration paths supplied"

2021-03-08 14:23:19.888 INFO    pyngrok.process: t=2021-03-08T14:23:19+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2021-03-08 14:23:19.892 INFO    pyngrok.process: t=2021-03-08T14:23:19+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2021-03-08 14:23:19.897 INFO    pyngrok.process: t=2021-03-08T14:23:19+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2021-03-08 14:23:20.050 INFO    pyngrok.process: t=2021-03-08T14:23:20+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2021-03-08 14:23:20.052 INFO    pyngrok.process: t=2021-03-08T14:23:20+0000 lvl=info msg="client session established" obj=csess id=6afd426fe607

2021-03-08 14:23:20.057 INFO    pyngrok.process: ngrok process has s

'http://6ea0c56ab36a.ngrok.io'